In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [2]:
from datetime import date, timedelta

start = '2021-06-01'
end = '2022-05-27'

start_date = date.fromisoformat(start)
end_date = date.fromisoformat(end)

date_range = [
    (end_date - timedelta(days=i)).strftime('%d-%m-%Y')
    for i
    in range((end_date - start_date).days)
]

In [4]:
cgk_mk_data = {
    'date':[],
    'price':[],
    'mc':[],
    'circ_supply':[]
}

for date in date_range:
    cgk_mk_data['date'].append(date)

    try:
        response_data = cg.get_coin_history_by_id(id='osmosis', date=date, localization=False)['market_data']
        cgk_mk_data['price'].append(response_data['current_price']['usd'])
        cgk_mk_data['mc'].append(response_data['market_cap']['usd'])
        cgk_mk_data['circ_supply'].append(response_data['market_cap']['usd'] / response_data['current_price']['usd'])
    except:
        cgk_mk_data['price'].append(None)
        cgk_mk_data['mc'].append(None)
        cgk_mk_data['circ_supply'].append(None)

In [9]:
df_cgk = pd.DataFrame(data=cgk_mk_data)
df_cgk.date = pd.to_datetime(df_cgk.date, format='%d-%m-%Y')
df_cgk.head()

,date,price,mc,circ_supply
0,2022-05-27,1.440695,5.488961e+08,3.809938e+08
1,2022-05-26,1.589280,6.030535e+08,3.794507e+08
2,2022-05-25,1.686672,6.384569e+08,3.785305e+08
3,2022-05-24,1.771739,6.690345e+08,3.776146e+08
4,2022-05-23,1.793356,6.759319e+08,3.769090e+08


In [10]:
df_flip = pd.read_json('https://node-api.flipsidecrypto.com/api/v2/queries/eb582ce9-3ade-46fe-b1d4-610c39ab82eb/data/latest')
df_flip.columns = df_flip.columns.str.lower()
df_flip.head()

,date,label_action,osmo
0,2022-01-16,STAKE,390309.507747
1,2022-01-06,STAKE,393338.485638
2,2022-04-10,STAKE,152698.353314
3,2022-04-17,STAKE,378511.787042
4,2022-04-20,STAKE,203255.050618


In [15]:
df = pd.merge(left=df_flip, right=df_cgk, on='date')\
    .sort_values(by=['date'], ascending=False)\
    .reset_index(drop=True)
df.head()

,date,label_action,osmo,price,mc,circ_supply
0,2022-05-27,UNSTAKE,1568.503222,1.440695,5.488961e+08,3.809938e+08
1,2022-05-27,STAKE,34363.512053,1.440695,5.488961e+08,3.809938e+08
2,2022-05-26,STAKE,612734.585593,1.589280,6.030535e+08,3.794507e+08
3,2022-05-26,UNSTAKE,91090.364694,1.589280,6.030535e+08,3.794507e+08
4,2022-05-25,UNSTAKE,64328.907407,1.686672,6.384569e+08,3.785305e+08
